This notebook details the process of identifying and tracking the values of tensors in a given network with an example using Mask RCNN.

In order to run this notebook on EC2, ssh into your instance with the command

```ssh -i /your/ec2/keypair -L localhost:8888:localhost:8888 -L localhost:6006:localhost:6006 ec2-user@ip
nohup jupyter notebook --no-browser --ip=0.0.0.0 > notebook.log &
tensorboard -logdir ~/logs
```

This notebook is broken into _ sections. First, we generate a small dataset consisting of a single image from the coco data. We then look at how to track the tensors within Mask RCNN using that image. Finally, we track the gradients that backpropogate through the network.

### Generate a single image dataset

Mask RCNN isn't setup to deal with a single image, so we instead create a small dataset that's just the same image repeated 10 times.

Randomly select an image from the COCO data, and move it to a new directory.

In [1]:
import os
import shutil
import numpy as np
import json
from copy import copy 

DATA_DIR = "/home/ec2-user/data/"
SINGLE_DATA = "/home/ec2-user/single_data/"
COUNT = 10

shutil.rmtree(SINGLE_DATA, ignore_errors=True)

os.makedirs(SINGLE_DATA, exist_ok=True)
os.makedirs(os.path.join(SINGLE_DATA, "train2017"), exist_ok=True)
os.makedirs(os.path.join(SINGLE_DATA, "annotations"), exist_ok=True)

image_dir = os.path.join(DATA_DIR, "train2017")

image = np.random.choice(os.listdir(image_dir), size=1)[0]

for num in range(1, COUNT+1):
    shutil.copy(os.path.join(image_dir, image), os.path.join(SINGLE_DATA, "train2017", 
                                                             image.split('.')[0]+str(num)+'.jpg'))

In [2]:
import matplotlib.pyplot as plt
plt.imshow(plt.imread(os.path.join(SINGLE_DATA, 
                        "train2017", 
                        np.random.choice(os.listdir(os.path.join(SINGLE_DATA, "train2017"))))))

Add the labels and mask annotations from the COCO data. We can just grab all of them

In [3]:
annotation_dir = os.path.join(DATA_DIR, "annotations")
'''
shutil.copy(os.path.join(annotation_dir, 'instances_val2017.json'),
            os.path.join(SINGLE_DATA, "annotations", "instances_val2017.json"))

shutil.copy(os.path.join(annotation_dir, 'captions_val2017.json'),
            os.path.join(SINGLE_DATA, "annotations", "captions_val2017.json"))

shutil.copy(os.path.join(annotation_dir, 'person_keypoints_val2017.json'),
            os.path.join(SINGLE_DATA, "annotations", "person_keypoints_val2017.json"))'''

shutil.copytree(os.path.join(DATA_DIR, 'val2017'),
            os.path.join(SINGLE_DATA, "val2017"))

'/home/ec2-user/single_data/val2017'

Create a new small set of annotations that match the filenames for the single image.

In [4]:
with open(os.path.join(annotation_dir, 'instances_train2017.json')) as infile:
    instances = json.loads(infile.readline())
    
image_info = [i for i in instances['images'] if i['file_name'] == image]
image_ids = [i['id'] for i in image_info]
instances['annotations'] = [i for i in instances['annotations'] if i['image_id'] in image_ids]
instances['images'] = [i for i in instances['images'] if i['id'] in image_ids]

images = []
annotations = []
for i in range(COUNT):
    images.append(copy(instances['images'][0]))
    images[i]['file_name'] = image.split('.')[0]+str(i+1) + '.jpg'
    images[i]['id'] = int(str(images[i]['id']) + str(i+1))
    annotations.append(copy(instances['annotations'][0]))
    annotations[i]['image_id'] = int(str(annotations[i]['image_id']) + str(i+1))

instances['images'] = images
instances['annotations'] = annotations


with open(os.path.join(SINGLE_DATA, "annotations", "instances_train2017.json"), 'w') as outfile:
    outfile.write(json.dumps(instances))
    
with open(os.path.join(annotation_dir, 'captions_train2017.json')) as infile:
    captions = json.loads(infile.readline())

captions['annotations'] = [i for i in captions['annotations'] if i['image_id'] in image_ids]
captions['images'] = [i for i in captions['images'] if i['id'] in image_ids]

images = []
annotations = []
for i, an_image in enumerate(images):
    images.append(copy(captions['images'][0]))
    images[i]['file_name'] = images[i]['file_name'].split('.')[0] + str(i+1) + '.jpg'
    images[i]['id'] = int(str(images[i]['id']) + str(i+1))

captions['images'] = images

with open(os.path.join(SINGLE_DATA, "annotations", "captions_train2017.json"), 'w') as outfile:
    outfile.write(json.dumps(captions))

with open(os.path.join(annotation_dir, 'person_keypoints_train2017.json')) as infile:
    keypoints = json.loads(infile.readline())

keypoints['annotations'] = [i for i in keypoints['annotations'] if i['image_id'] in image_ids]
keypoints['images'] = [i for i in keypoints['images'] if i['id'] in image_ids]

images = [copy(keypoints['images'][0]) for _ in range(COUNT)]
for i, an_image in enumerate(images):
    images[i]['file_name'] = images[i]['file_name'].split('.')[0] + str(i+1) + '.jpg'
    images[i]['id'] = int(str(images[i]['id']) + str(i+1))

with open(os.path.join(SINGLE_DATA, "annotations", "person_keypoints_train2017.json"), 'w') as outfile:
    outfile.write(json.dumps(keypoints))


Read in tensorpack packages. 

In [5]:
import sys
#os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
import tensorflow as tf
import tqdm
import tensorpack.utils.viz as tpviz
from tensorpack import *
from tensorpack.tfutils.common import get_tf_version_tuple
sys.path.append('/home/ec2-user/mask-rcnn-tensorflow/MaskRCNN')
from model.generalized_rcnn import ResNetFPNModel
from config import finalize_configs, config as cfg
from eval import DetectionResult, predict_image, multithread_predict_dataflow, EvalCallback
from performance import ThroughputTracker, humanize_float
from data import get_eval_dataflow, get_train_dataflow, get_batch_train_dataflow


/home/ec2-user/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/anaconda3/lib/python3.6/site-packages/te

Load model and configuration.

In [6]:
MODEL = ResNetFPNModel(True)

In [7]:
#cfg.DATA.BASEDIR = '/home/ec2-user/data'
cfg.DATA.BASEDIR = '/home/ec2-user/single_data'
#cfg.DATA.BASEDIR = '/home/ec2-user/small_data'
cfg.BACKBONE.WEIGHTS = '/home/ec2-user/data/pretrained-models/ImageNet-R50-AlignPadding.npz'
#cfg.MODE_FPN=True
#cfg.FPN.NORM = 'GN'
#cfg.TRAIN.BATCH_SIZE_PER_GPU = 4
tf.set_random_seed(cfg.TRAIN.SEED)
fix_rng_seed(cfg.TRAIN.SEED)
np.random.seed(cfg.TRAIN.SEED)

In [8]:
train_dataflow = get_batch_train_dataflow(cfg.TRAIN.BATCH_SIZE_PER_GPU)
finalize_configs(is_training=True)

In train dataflow
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[1014 07:06:49 @dataset.py:50] Instances loaded from /home/ec2-user/single_data/annotations/instances_train2017.json.


100%|██████████| 10/10 [00:00<00:00, 9006.45it/s]

[1014 07:06:49 @timer.py:50] Load annotations for train2017 finished, time:0.0030sec.
Done loading roidbs
[1014 07:06:49 @data.py:509] Filtered 0 images which contain no non-crowd groudtruth boxes. Total #images for training: 10


Batching roidbs
Done batching roidbs
[1014 07:06:49 @config.py:280] Config: ------------------------------------------
{'BACKBONE': {'FREEZE_AFFINE': False,
              'FREEZE_AT': 2,
              'NORM': 'FreezeBN',
              'RESNET_NUM_BLOCKS': [3, 4, 6, 3],
              'STRIDE_1X1': False,
              'TF_PAD_MODE': False,
              'WEIGHTS': '/home/ec2-user/data/pretrained-models/ImageNet-R50-AlignPadding.npz'},
 'DATA': {'BASEDIR': '/home/ec2-user/single_data',
          'CLASS_NAMES': ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                          'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
                          'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
                          'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
                    

In [9]:
session_init = get_model_loader(cfg.BACKBONE.WEIGHTS)

W1014 07:06:49.936255 139694008874816 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.6/site-packages/tensorpack-0.9.0.1-py3.6.egg/tensorpack/tfutils/sessinit.py:259: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.



In [10]:
traincfg = TrainConfig(
            model=MODEL,
            data=QueueInput(train_dataflow),
            steps_per_epoch=1,
            max_epoch=1,
            session_init=session_init,
            session_config=None,
            starting_epoch=cfg.TRAIN.STARTING_EPOCH
        )

In [11]:
trainer = SimpleTrainer()


At this point, we have our data and the model is setup. In order to export a gradient from the graph, we need to add a print statement to the tensor we want to track. For example, say we want to export the second to last layer of the backbone network (c4). Add this import to the top of the backbone.py file:

```from performance import print_runtime_tensor, print_runtime_tensor_loose_branch```

The, just after the c4 tensor is created in the network, add this line:

```c4 = print_runtime_tensor("tensor_c4_forward", c4)```

Similarly, say we want to output a list of tensors. Perhaps the full output of the backbone (p23456). We can use something like:

```p23456 = [print_runtime_tensor("tensor_p23456_{}_forward".format(i), j) for i,j in enumerate(p23456)]```

On the other hand, we might want to see a tensor that isn't actually used later in the graph, which means it wouldn't normally execute such that we can output it. This can be dome using the 

```print_runtime_tensor_loose_branch```

For this, you need a downstream trigger tesnor to force the print of the tensor of interest. Say we have a tensor `t5` that isn't used in the graph, but `t1` is. We can print `t5` with:

```t1 = print_runtime_tensor_loose_branch("tensor_t5_forward", t5, trigger_tensor=t1)```

Finally, say we want to print the gradients of the backwards pass. This is a little more complicated. Add this gradient printer class to the generalized_rcnn.py file:

```
class GradientPrinter(tf.train.Optimizer):
    def __init__(self, opt):
        self.opt = opt
    def compute_gradients(self, *args, **kwargs):
        return self.opt.compute_gradients(*args, **kwargs)
    def apply_gradients(self, gradvars, global_step=None, name=None):
        old_grads, v = zip(*gradvars)
        old_grads = [print_runtime_tensor("tensor_{}_backward".format(i.name), j) for i,j in zip(v, old_grads)]
        for i in v:
            print("gradient_name: {}".format(i.name))
        gradvars = list(zip(old_grads, v))
        return self.opt.apply_gradients(gradvars, global_step, name)
```

Inside the detection model class, modify the optimizer to pass through the gradient printer.

```
opt = tf.train.MomentumOptimizer(lr, 0.9)
opt = GradientPrinter(opt)
```

Once the print function has been added, run the paragraph below with the capture magic function to catch the printed output.

In [12]:
%%capture cap_out --no-stderr
launch_train_with_config(traincfg, trainer)

[1014 07:06:51 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...


W1014 07:06:51.453391 139694008874816 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.6/site-packages/tensorpack-0.9.0.1-py3.6.egg/tensorpack/tfutils/summary.py:266: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.



[1014 07:06:51 @trainers.py:49] Building graph for a single training tower ...
[1014 07:06:51 @registry.py:127] conv0 input: [None, 3, None, None]


W1014 07:06:51.495776 139694008874816 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/lib/python3.6/site-packages/tensorpack-0.9.0.1-py3.6.egg/tensorpack/models/conv2d.py:81: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.

W1014 07:06:53.834445 139694008874816 ag_logging.py:145] Entity <function BatchNorm at 0x7f0cb1497840> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <function BatchNorm at 0x7f0cb1497840>: AssertionError: If not matching a CFG node, must be a block statement: <gast.gast.ImportFrom object at 0x7f0c867fae10>


[1014 07:06:53 @batch_norm.py:166] WRN [BatchNorm] Using moving_mean/moving_variance in training.


...

[1014 07:07:18 @monitor.py:259] WRN logger directory was not set. Ignore TFEventWriter.
[1014 07:07:18 @monitor.py:300] WRN logger directory was not set. Ignore JSONWriter.
[1014 07:07:18 @model_utils.py:66] Trainable Variables: 
name                        shape                   dim
--------------------------  -----------------  --------
fpn/lateral_1x1_c2/W:0      [1, 1, 256, 256]      65536
fpn/lateral_1x1_c2/b:0      [256]                   256
fpn/lateral_1x1_c3/W:0      [1, 1, 512, 256]     131072
fpn/lateral_1x1_c3/b:0      [256]                   256
fpn/lateral_1x1_c4/W:0      [1, 1, 1024, 256]    262144
fpn/lateral_1x1_c4/b:0      [256]                   256
fpn/lateral_1x1_c5/W:0      [1, 1, 2048, 256]    524288
fpn/lateral_1x1_c5/b:0      [256]                   256
fpn/posthoc_3x3_p2/W:0      [3, 3, 256, 256]     589824
fpn/posthoc_3x3_p2/b:0      [256]                   256
fpn/posthoc_3x3_p3/W:0      [3, 3, 256, 256]     589824
fpn/posthoc_3x3_p3/b:0      [256]         

W1014 07:07:26.060058 139694008874816 deprecation.py:323] From /home/ec2-user/anaconda3/lib/python3.6/site-packages/tensorpack-0.9.0.1-py3.6.egg/tensorpack/tfutils/varmanip.py:108: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


[1014 07:07:26 @varmanip.py:104] WRN Variable group3/block2/conv3/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1014 07:07:26 @varmanip.py:104] WRN Variable group3/block2/conv2/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1014 07:07:26 @varmanip.py:104] WRN Variable group3/block2/conv1/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1014 07:07:27 @varmanip.py:104] WRN Variable group1/block3/conv1/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1014 07:07:28 @varmanip.py:104] WRN Variable group1/block0/conv1/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1014 07:07:29 @varmanip.py:104] WRN Variable group2/block4/conv2/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1014 07:07:29 @

100%|##########|1/1[00:25<00:00, 0.04it/s]

[1014 07:08:15 @base.py:291] Epoch 1 (global_step 1) finished, time:25.5 seconds.
[1014 07:08:15 @monitor.py:469] QueueInput/queue_size: 42
[1014 07:08:15 @monitor.py:469] boxclass_losses/box_loss: 0.0033458
[1014 07:08:15 @monitor.py:469] boxclass_losses/label_loss: 4.5305
[1014 07:08:15 @monitor.py:469] boxclass_losses/label_metrics/accuracy: 0
[1014 07:08:15 @monitor.py:469] boxclass_losses/label_metrics/false_negative: 0
[1014 07:08:15 @monitor.py:469] boxclass_losses/label_metrics/fg_accuracy: 0
[1014 07:08:15 @monitor.py:469] boxclass_losses/num_fg_label: 2
[1014 07:08:15 @monitor.py:469] learning_rate: 0.003
[1014 07:08:15 @monitor.py:469] maskrcnn_loss/accuracy: 0.38329
[1014 07:08:15 @monitor.py:469] maskrcnn_loss/fg_pixel_ratio: 0.66709
[1014 07:08:15 @monitor.py:469] maskrcnn_loss/maskrcnn_loss: 0.97576
[1014 07:08:15 @monitor.py:469] maskrcnn_loss/pos_accuracy: 0.10714
[1014 07:08:15 @monitor.py:469] multilevel_roi_align/fpn_map_rois_to_levels/num_roi_level2: 417
[1014 07:0

The output will be stored in the `cap_out` string, but also includes a lot of other junk. To clean it up, use regular expressions and a literal eval.

Note that the examples above all follow the format `tensor_something_forward` or `tensor_something_backward` this is to make cleaning up the string easier.

The example below assumes the tensors are named something like `tensor_some_tensor_name_backwards`

In [13]:
from ast import literal_eval
import json
import re

In [14]:
result = cap_out.stdout.split('[runtime_tensor] tensor_')[1:]
result = {i.split('_backward ')[0]: literal_eval(re.sub('\s+', ', ', i.split('_backward ')[1].strip())) for i in result}

[1014 07:08:31 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.


In [16]:
len(result)

42

In [17]:
result.keys()

dict_keys(['rpn/class/b:0', 'rpn/class/W:0', 'rpn/box/b:0', 'fastrcnn/outputs/class/b:0', 'rpn/conv0/b:0', 'fastrcnn/outputs/box/b:0', 'rpn/box/W:0', 'fastrcnn/fc7/b:0', 'fastrcnn/fc6/b:0', 'maskrcnn/conv/b:0', 'maskrcnn/deconv/b:0', 'maskrcnn/fcn3/b:0', 'fpn/posthoc_3x3_p4/b:0', 'maskrcnn/fcn2/b:0', 'maskrcnn/conv/W:0', 'fastrcnn/outputs/class/W:0', 'maskrcnn/fcn1/b:0', 'maskrcnn/fcn0/b:0', 'maskrcnn/deconv/W:0', 'fastrcnn/outputs/box/W:0', 'fpn/lateral_1x1_c5/b:0', 'rpn/conv0/W:0', 'maskrcnn/fcn1/W:0', 'fpn/lateral_1x1_c4/W:0', 'fpn/posthoc_3x3_p4/W:0', 'maskrcnn/fcn0/W:0', 'fpn/lateral_1x1_c3/b:0', 'fpn/posthoc_3x3_p5/b:0', 'fpn/lateral_1x1_c2/b:0', 'fpn/lateral_1x1_c2/W:0', 'fpn/lateral_1x1_c3/W:0', 'fpn/lateral_1x1_c4/b:0', 'maskrcnn/fcn3/W:0', 'fpn/posthoc_3x3_p3/b:0', 'fpn/posthoc_3x3_p2/b:0', 'fastrcnn/fc7/W:0', 'fpn/posthoc_3x3_p5/W:0', 'maskrcnn/fcn2/W:0', 'fpn/posthoc_3x3_p3/W:0', 'fpn/lateral_1x1_c5/W:0', 'fpn/posthoc_3x3_p2/W:0', 'fastrcnn/fc6/W:0'])

In [18]:
result['rpn/class/b:0']

[0.155761719, 0.128295898, 0.165771484]